In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

def fetch_weather_data(lat=52.52, long=13.41,timestamp=None,dateStart=None,dateEnd=None):

    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"

    params = {}

    if dateStart is not None:
        params = {
        "latitude": lat,
        "longitude": long,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "surface_pressure"],
        "start_date":dateStart,
        "end_date":dateEnd,
        "timezone": "auto"
    }
    else:
        params = {
        "latitude": lat,
        "longitude": long,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "surface_pressure"],
        "forecast_days":2,
        "timezone": "auto"
    }

    
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation_probability = hourly.Variables(4).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation_probability"] = hourly_precipitation_probability
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["surface_pressure"] = hourly_surface_pressure

    hourly_dataframe = pd.DataFrame(data = hourly_data)

    if timestamp is None:
        #print("calling none")
        return hourly_dataframe
    else:
       #Find station based on timestamp
        station_row = hourly_dataframe.loc[hourly_dataframe['date'] == timestamp].copy()

        #update timestamp 
        station_row['date'] = pd.to_datetime(station_row['date']).dt.strftime('%Y-%m-%d %H:%M:%S')
        #return station row
        return station_row

In [2]:
import requests
import csv
import os
import pandas as pd
from datetime import datetime


URL = "https://api.jcdecaux.com/vls/v1/stations?contract=maribor&apiKey=5e150537116dbc1786ce5bec6975a8603286526b"
#urrent_dir = os.path.dirname(os.path.abspath(__file__))
#mbajk_dir = os.path.join(current_dir, 'data', 'raw')

#TODO update path
#CURR_DIR = os.path.dirname(os.path.abspath(__file__))
#SAVE_WEATHER_DIR = os.path.join(CURR_DIR, '..', '..', 'data', 'raw','weather', 'weather.csv')
#SAVE_MBAJK_DIR = os.path.join(CURR_DIR, '..', '..', 'data', 'raw', 'fetch_mbajk.csv')

SAVE_FILE = r'C:\Users\Jan\Desktop\FERI\IPT\2-SEMESTER\IIS\nove_vaje\IIS\data\raw\fetch_mbajk.csv'
SAVE_WEATHER_FILE = r'C:\Users\Jan\Desktop\FERI\IPT\2-SEMESTER\IIS\nove_vaje\IIS\data\raw\weather\weather.csv'


def fetch_data(api_url):
    try:
        res = requests.get(api_url)
        if res.status_code == 200:
            data = res.json()
            return data
        else: 
            print(f"Error {res.status_code} - {res.text}")
            return None
    except requests.RequestException as e:
        print(f"Error: {e}")
        return None

data = fetch_data(URL)
if data is not None:

    print(f"Data fetched successfully, {len(data)} records found")

    
    if not os.path.exists(SAVE_WEATHER_FILE):

    
        station_weather = fetch_weather_data()
        station_weather["station_name"] = ""
        header = station_weather.keys()
        
        

        with open(SAVE_WEATHER_FILE,'w', newline='', encoding='utf-8',errors='replace') as weatherFile:
            weather_writer = csv.writer(weatherFile, lineterminator='\n')
            weather_writer.writerow(header)


    #check if file already has data 

    print(data)

    df = pd.DataFrame(data)
    
    df = df[['last_update']].sort_values(by='last_update', ascending=True)

    df['last_update'] = pd.to_datetime(df['last_update'], unit='ms')
    df['last_update'] = df['last_update'].dt.round('H')

    print(f"first time is {df.iloc[0]}" )

    df['last_update'] = df['last_update'].dt.date

    #print first row
    first = df.iloc[0]
    last = df.iloc[-1]

    date_before_first_day = first - pd.Timedelta(days=1)
    date_after_last_day = first + pd.Timedelta(days=1)

    
    for station in data:
        timestamp = pd.to_datetime(station['last_update'], unit='ms')
        rounded_timestamp = timestamp.round('H')
        formatted_timestamp = rounded_timestamp.strftime('%Y-%m-%d %H:%M:%S+00:00')

        #  print("timestamp is", formatted_timestamp)
        station_weather = fetch_weather_data(station['position']['lat'], station['position']['lng'], formatted_timestamp,date_before_first_day,date_after_last_day)
        station_weather['name'] = station['name']

        station['last_update']=formatted_timestamp
        station_weather['date']=formatted_timestamp

        print(f"station {station['name']}with timestamp {formatted_timestamp} and weather {station_weather['date']}")

# try to write to weather.csv file
        try:
            station_weather.to_csv(SAVE_WEATHER_FILE, mode='a', header=False, index=False)
        except:
            print("Errro writting to weather.csv ")

# write to mbajk.csv file        
    with open(SAVE_FILE, 'a', newline='', encoding='utf-8',errors='replace') as csvfile:
        csv_writer = csv.writer(csvfile, lineterminator='\n')
# Če je datoteka prazna, dodaj header
        if os.path.getsize(SAVE_FILE) == 0:  
            header = data[0].keys()
            csv_writer.writerow(header)

        for row in data:
            csv_writer.writerow(row.values())
        
        print(f"Data succesfully fetched and saved to {SAVE_FILE}")
else: 
    print("Error occurred, data not available")


Data fetched successfully, 29 records found
[{'number': 24, 'contract_name': 'maribor', 'name': 'VZAJEMNA, VARUH ZDRAVJA - BETNAVSKA C.', 'address': 'Betnavska c. 83', 'position': {'lat': 46.54199, 'lng': 15.63831}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 1, 'available_bikes': 19, 'status': 'OPEN', 'last_update': 1713627698000}, {'number': 10, 'contract_name': 'maribor', 'name': 'TELEMACH - GLAVNI TRG - STARI PERON', 'address': 'Glavni trg 20', 'position': {'lat': 46.557328, 'lng': 15.646833}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 13, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1713627905000}, {'number': 9, 'contract_name': 'maribor', 'name': 'NKBM - TRG LEONA ŠTUKLJA', 'address': 'Ulica slovenske osamosvojitve 5', 'position': {'lat': 46.558519, 'lng': 15.649072}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 18, 'available_bikes': 2, 'status': 'OPEN', 'last_update':